In [1]:
import zipfile
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
zip_file = 'Dataset.zip'  # Path to your zip file
unzip_dir = 'dataset'  # Directory where the files will be extracted

try:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(unzip_dir)
except zipfile.BadZipFile:
    print("Error: The zip file might have unusual formatting or features.")
    print("Trying to extract using 'shutil' module as an alternative...")
    import shutil
    shutil.unpack_archive(zip_file, unzip_dir, 'zip')  # Use shutil to extract
    print("Extraction successful using shutil.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


In [3]:
data = []

# Adjusting paths based on your new directory structure
unzip_dir = 'dataset'  # This is the base folder where 'Dataset' is located
healthy_folder = os.path.join(unzip_dir, 'Dataset/Healthy')  # Path to 'Healthy' folder
unhealthy_folder = os.path.join(unzip_dir, 'Dataset/Unhealthy')  # Path to 'Unhealthy' folder

# Check if the folders exist (just for safety)
if not os.path.exists(healthy_folder):
    print(f"Healthy folder does not exist: {healthy_folder}")
if not os.path.exists(unhealthy_folder):
    print(f"Unhealthy folder does not exist: {unhealthy_folder}")

# Iterate through both folders and create labels
for label, folder in [('healthy', healthy_folder), ('unhealthy', unhealthy_folder)]:
    if os.path.exists(folder):  # Proceed only if the folder exists
        for filename in os.listdir(folder):
            if filename.endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
                image_path = os.path.join(folder, filename)
                data.append([image_path, label])

# Step 3: Load into a pandas DataFrame
df = pd.DataFrame(data, columns=['image_path', 'label'])

# Step 4: Split the dataset into train and test sets (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Optionally, you can save these datasets to CSVs or use them in further processing
train_df.to_csv('train_dataset.csv', index=False)
test_df.to_csv('test_dataset.csv', index=False)

# Show the first few rows of the training and test data
print("Training Data:")
print(train_df.head())

print("Test Data:")
print(test_df.head())

Training Data:
                                  image_path      label
873  dataset/Dataset/Unhealthy/IMG_4217.jpeg  unhealthy
825  dataset/Dataset/Unhealthy/IMG_3249.jpeg  unhealthy
618    dataset/Dataset/Healthy/IMG_3267.jpeg    healthy
879  dataset/Dataset/Unhealthy/IMG_4364.jpeg  unhealthy
590    dataset/Dataset/Healthy/IMG_3942.jpeg    healthy
Test Data:
                                  image_path      label
423    dataset/Dataset/Healthy/IMG_3851.jpeg    healthy
548    dataset/Dataset/Healthy/IMG_4399.jpeg    healthy
959  dataset/Dataset/Unhealthy/IMG_4341.jpeg  unhealthy
587    dataset/Dataset/Healthy/IMG_3554.jpeg    healthy
827  dataset/Dataset/Unhealthy/IMG_4471.jpeg  unhealthy


In [4]:
import os
import torch
import pandas as pd
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets, models
from sklearn.model_selection import train_test_split
from efficientnet_pytorch import EfficientNet

# Step 1: Load the train and test datasets
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')

# Step 2: Custom Dataset Class to Load Images
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        label = 1 if self.dataframe.iloc[idx, 1] == 'healthy' else 0
        image = datasets.folder.default_loader(img_path)
        
        if self.transform:
            image = self.transform(image)

        return image, label

# Step 3: Define transformations (for training and testing)
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # EfficientNet expects 224x224 input size
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

# Step 4: Create DataLoaders
train_dataset = CustomImageDataset(dataframe=train_df, transform=train_transforms)
test_dataset = CustomImageDataset(dataframe=test_df, transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Step 5: Load EfficientNet Model (Pre-trained)
model = EfficientNet.from_pretrained('efficientnet-b0')

# Modify the final layer for binary classification (2 classes: healthy, unhealthy)
model._fc = nn.Linear(in_features=model._fc.in_features, out_features=2)

# Step 6: Set device for training (CUDA if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Step 7: Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Step 8: Train the Model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

# Step 9: Evaluate the Model on Test Set
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /Users/shafayat/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:07<00:00, 2.97MB/s]


Loaded pretrained weights for efficientnet-b0
Epoch [1/10], Loss: 0.5924, Accuracy: 74.22%
Epoch [2/10], Loss: 0.3683, Accuracy: 89.90%
Epoch [3/10], Loss: 0.2337, Accuracy: 93.50%
Epoch [4/10], Loss: 0.1579, Accuracy: 95.12%
Epoch [5/10], Loss: 0.1148, Accuracy: 96.40%
Epoch [6/10], Loss: 0.0817, Accuracy: 97.33%
Epoch [7/10], Loss: 0.0648, Accuracy: 98.14%
Epoch [8/10], Loss: 0.0630, Accuracy: 98.37%
Epoch [9/10], Loss: 0.0476, Accuracy: 98.37%
Epoch [10/10], Loss: 0.0430, Accuracy: 98.84%
Test Accuracy: 96.76%


In [5]:
# Save the model's state_dict (weights)
torch.save(model.state_dict(), 'efficientnet_model.pth')
print("Model saved successfully!")


Model saved successfully!


In [6]:
# Recreate the model architecture
model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = nn.Linear(in_features=model._fc.in_features, out_features=2)  # Modify for binary classification

# Load the saved state_dict
model.load_state_dict(torch.load('efficientnet_model.pth'))
model = model.to(device)  # Make sure to move the model to the correct device (CPU or GPU)
print("Model loaded successfully!")


Loaded pretrained weights for efficientnet-b0
Model loaded successfully!


In [7]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score, average_precision_score
import numpy as np
import torch

# Step 10: Compute all necessary metrics (accuracy, precision, recall, F1 score, ROC AUC, PR AUC, confusion matrix)
model.eval()  # Set the model to evaluation mode
y_true = []
y_pred = []
y_prob = []  # To store predicted probabilities for ROC AUC and PR AUC

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
        
        # Store the predicted probabilities for ROC AUC and PR AUC (softmax output)
        prob = torch.softmax(outputs, dim=1)[:, 1]  # Probabilities for the 'healthy' class
        y_prob.extend(prob.cpu().numpy())

# Convert to numpy arrays for easier handling with sklearn
y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_prob = np.array(y_prob)

# Accuracy
accuracy = np.sum(y_pred == y_true) / len(y_true)

# Precision, Recall, F1 Score
precision = precision_score(y_true, y_pred, average='binary')
recall = recall_score(y_true, y_pred, average='binary')
f1 = f1_score(y_true, y_pred, average='binary')

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Classification Report
class_report = classification_report(y_true, y_pred)

# ROC AUC (Area Under the ROC Curve)
roc_auc = roc_auc_score(y_true, y_prob)

# PR AUC (Area Under the Precision-Recall Curve)
pr_auc = average_precision_score(y_true, y_prob)

# Print the results
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'ROC AUC: {roc_auc:.4f}')
print(f'PR AUC: {pr_auc:.4f}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')


Accuracy: 0.9676
Precision: 0.9651
Recall: 0.9940
F1 Score: 0.9794
ROC AUC: 0.9907
PR AUC: 0.9971
Confusion Matrix:
[[ 43   6]
 [  1 166]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.88      0.92        49
           1       0.97      0.99      0.98       167

    accuracy                           0.97       216
   macro avg       0.97      0.94      0.95       216
weighted avg       0.97      0.97      0.97       216



In [9]:
import os
import torch
import pandas as pd
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets, models
from sklearn.model_selection import train_test_split

# Step 1: Load the train and test datasets
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')

# Step 2: Custom Dataset Class to Load Images
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        label = 1 if self.dataframe.iloc[idx, 1] == 'healthy' else 0
        image = datasets.folder.default_loader(img_path)
        
        if self.transform:
            image = self.transform(image)

        return image, label

# Step 3: Define transformations (for training and testing)
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet expects 224x224 input size
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

# Step 4: Create DataLoaders
train_dataset = CustomImageDataset(dataframe=train_df, transform=train_transforms)
test_dataset = CustomImageDataset(dataframe=test_df, transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Step 5: Load ResNet Model (Pre-trained)
model = models.resnet50(pretrained=True)  # Use ResNet-18, or you can try resnet34, resnet50, etc.

# Modify the final layer for binary classification (2 classes: healthy, unhealthy)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # Output 2 classes

# Step 6: Set device for training (CUDA if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Step 7: Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Step 8: Train the Model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

# Step 9: Evaluate the Model on Test Set
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/shafayat/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:25<00:00, 4.04MB/s]


Epoch [1/10], Loss: 0.3288, Accuracy: 86.18%
Epoch [2/10], Loss: 0.1428, Accuracy: 95.70%
Epoch [3/10], Loss: 0.0931, Accuracy: 97.44%
Epoch [4/10], Loss: 0.0649, Accuracy: 97.33%
Epoch [5/10], Loss: 0.0533, Accuracy: 97.79%
Epoch [6/10], Loss: 0.0540, Accuracy: 97.56%
Epoch [7/10], Loss: 0.0418, Accuracy: 98.95%
Epoch [8/10], Loss: 0.0479, Accuracy: 97.79%
Epoch [9/10], Loss: 0.0455, Accuracy: 98.14%
Epoch [10/10], Loss: 0.0328, Accuracy: 98.61%
Test Accuracy: 97.69%


In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score, average_precision_score
import numpy as np
import torch

# Step 10: Compute all necessary metrics (accuracy, precision, recall, F1 score, ROC AUC, PR AUC, confusion matrix)
model.eval()  # Set the model to evaluation mode
y_true = []
y_pred = []
y_prob = []  # To store predicted probabilities for ROC AUC and PR AUC

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
        
        # Store the predicted probabilities for ROC AUC and PR AUC (softmax output)
        prob = torch.softmax(outputs, dim=1)[:, 1]  # Probabilities for the 'healthy' class
        y_prob.extend(prob.cpu().numpy())

# Convert to numpy arrays for easier handling with sklearn
y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_prob = np.array(y_prob)

# Accuracy
accuracy = np.sum(y_pred == y_true) / len(y_true)

# Precision, Recall, F1 Score
precision = precision_score(y_true, y_pred, average='binary')
recall = recall_score(y_true, y_pred, average='binary')
f1 = f1_score(y_true, y_pred, average='binary')

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Classification Report
class_report = classification_report(y_true, y_pred)

# ROC AUC (Area Under the ROC Curve)
roc_auc = roc_auc_score(y_true, y_prob)

# PR AUC (Area Under the Precision-Recall Curve)
pr_auc = average_precision_score(y_true, y_prob)

# Print the results
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'ROC AUC: {roc_auc:.4f}')
print(f'PR AUC: {pr_auc:.4f}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')


Accuracy: 0.9769
Precision: 0.9765
Recall: 0.9940
F1 Score: 0.9852
ROC AUC: 0.9890
PR AUC: 0.9962
Confusion Matrix:
[[ 45   4]
 [  1 166]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.92      0.95        49
           1       0.98      0.99      0.99       167

    accuracy                           0.98       216
   macro avg       0.98      0.96      0.97       216
weighted avg       0.98      0.98      0.98       216



In [11]:
# Save the model's state_dict
torch.save(model.state_dict(), 'resnet50_model.pth')
print("model saved successfully")

model saved successfully


In [12]:
model = models.resnet50(pretrained=False)

# Modify the final layer for binary classification (2 classes: healthy, unhealthy)
model.fc = torch.nn.Linear(in_features=model.fc.in_features, out_features=2)

# Step 2: Load the saved state_dict (weights), allowing for mismatch in final layer
model.load_state_dict(torch.load('resnet50_model.pth'), strict=False)
print("loaded successfully")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


RuntimeError: Error(s) in loading state_dict for ResNet:
	size mismatch for fc.weight: copying a param with shape torch.Size([2, 2048]) from checkpoint, the shape in current model is torch.Size([1000, 2048]).
	size mismatch for fc.bias: copying a param with shape torch.Size([2]) from checkpoint, the shape in current model is torch.Size([1000]).